In [ ]:
import cohere
import getpass
from langchain import PromptTemplate
from langchain.llms import Cohere
from google.colab import auth
from google.colab import drive

drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/Colab Notebooks/content.txt"

try:
  with open(file_path, "r", encoding="ISO-8859-1") as file:
     text_content = file.read()
  print("✅ File loaded successfully!")
except Exception as e:
  print("❌ Error loading file:", str(e))

COHERE_API_KEY = "4jZ1rr4LalaGrfHm3TIvImQpc4sD1JUnDF1cl26b"

cohere_llm = Cohere(cohere_api_key=COHERE_API_KEY, model="command")

template = """
You are an AI assistant helping to summarize and analyze a text document. Here is the document content:

{text}

🔹 Summary:
-   Provide a concise summary of the document.

🔹 Key Takeaways:
-   List 3 important points from the text.

🔹 Sentiment Analysis:
-   Determine if the sentiment of the document is Positive, Negative, or Neutral. """
prompt_template = PromptTemplate(input_variables=["text"], template=template)

formatted_prompt = prompt_template.format(text=text_content)

response = cohere_llm.predict(formatted_prompt)

print("\n📌 **Formatted Output** 📌")
print(response)



Mounted at /content/drive
✅ File loaded successfully!


<ipython-input-3-2a310d2bc1c7>:20: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  cohere_llm = Cohere(cohere_api_key=COHERE_API_KEY, model="command")
<ipython-input-3-2a310d2bc1c7>:39: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = cohere_llm.predict(formatted_prompt)



📌 **Formatted Output** 📌


Summary: This essay explores the evolution and impact of First-Person Shooter (FPS) games, a genre that has revolutionized the gaming industry. Beginning with pivotal titles like 'Wolfenstein 3D' and 'Doom,' FPS games have continually evolved with technological advancements, introducing narrative depth, multiplayer experiences, and increasingly sophisticated gameplay. The genre's impact extends beyond entertainment, shaping esports and influencing social interactions and competitive gaming cultures. FPS games will likely continue to be an influential force in the years to come. 

Key Takeaways: 

1. FPS games have thrived as one of the most influential gaming genres, marking significant progress since their early 1990s origins.
2. The immersive experience is a key differentiator, triggering a stronger sense of engagement and participation amongst players.
3. Multi-player and competitive gaming dynamics have played a critical role in the genre's broad appeal 

In [ ]:
# Install required libraries
#pip install langchain cohere wikipedia-api pydantic
#pip install langchain_community

# Import necessary libraries
from langchain import PromptTemplate, LLMChain
from langchain.llms import Cohere
from pydantic import BaseModel
from typing import Optional
import wikipediaapi
from IPython.display import display
import ipywidgets as widgets
# Step 1: Set up the Cohere API
import getpass
COHERE_API_KEY = "4jZ1rr4LalaGrfHm3TIvImQpc4sD1JUnDF1cl26b"
cohere_llm = Cohere(cohere_api_key=COHERE_API_KEY, model="command")

# Step 2: Download Indian Penal Code (IPC) summary from Wikipedia
def fetch_ipc_summary():
 user_agent = "IPCChatbot/1.0 (contact: myemail@example.com)"
 wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language='en')
 page = wiki_wiki.page("Indian Penal Code")

 if not page.exists():
   raise ValueError("The Indian Penal Code page does not exist on Wikipedia.")

 return page.text[:5000] # Limiting to first 5000 characters for brevity

ipc_content = fetch_ipc_summary()

# Step 3: Define a Pydantic model for structured responses
class IPCResponse(BaseModel):
 section: Optional[str]
 explanation: Optional[str]

# Step 4: Create a prompt template for the chatbot
prompt_template = PromptTemplate(
 input_variables=["question", "ipc_content"],
 template="""
 You are a legal assistant chatbot specialized in the Indian Penal Code (IPC).
 Refer to the following IPC document content to answer the user's query:

 {ipc_content}

 User Question: {question}

 Provide a detailed answer, mentioning the relevant section if applicable.
 """
 )
# Step 5: Create a function to generate IPC response using Cohere
def get_ipc_response(question: str) -> IPCResponse:
    # Format the prompt using the provided question and IPC content
    formatted_prompt = prompt_template.format(ipc_content=ipc_content, question=question)

    # Get the response from the Cohere model
    response = cohere_llm.predict(formatted_prompt)

    # Extracting and structuring the response
    if "Section" in response:
        section = response.split('Section')[1].split(':')[0].strip()
        explanation = response.split(':', 1)[-1].strip()
    else:
        section = None
        explanation = response.strip()

    return IPCResponse(section=section, explanation=explanation)


# Function to display the response
def display_response(response: IPCResponse):
    print(f"Section: {response.section if response.section else 'N/A'}")
    print(f"Explanation: {response.explanation}")

# Function to handle user input
def on_button_click(b):
    user_question = text_box.value
    try:
        response = get_ipc_response(user_question)
        display_response(response)
    except Exception as e:
        print(f"Error: {e}")

# Create text box and button for user input
text_box = widgets.Text(
    value='',
    placeholder='Ask about the Indian Penal Code',
    description='You:',
    disabled=False
)

button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='',
    tooltip='Click to ask a question about IPC',
    icon='legal'
)

button.on_click(on_button_click)

# Display the chatbot interface
display(text_box, button)

Text(value='', description='You:', placeholder='Ask about the Indian Penal Code')

Button(description='Ask', icon='legal', style=ButtonStyle(), tooltip='Click to ask a question about IPC')

Section: N/A
Explanation: According to the Indian Penal Code (IPC) 302, individuals who commit homicide are considered guilty of murder. whoruns the risk of punishment if found guilty. 

In addition to specifying the elements of the crime, the statute also outlines the circumstances under which the offense was committed and divides them into three categories. 

These categories differentiate between murders perpetrated through deliberate premeditation, reckless abandonment of self-defense, or during the commission of another crime. 

The statute also includes within its scope murders committed by concealment of birth, by forbidment of food or drink, by any willful act, or by any negligent act.

IPC 302 delineates the qualifications of accusers and accused, definitions of culpable mental state, the legal elements of murder, and the three categories of murder offenses. 

It is important to note that the IPC has since been replaced by the Bharatiya Nyaya Sanhita (BNS), come into effect on

In [ ]:
from IPython.display import display
import ipywidgets as widgets

# Function to display chatbot responses
def display_response(response: IPCResponse):
 print(f"Section: {response.section if response.section else 'N/A'}")
 print(f"Explanation: {response.explanation}")

# Function to handle user input
def on_button_click(b):
 user_question = text_box.value
try:
 response = get_ipc_response(user_question)
 display_response(response)
except Exception as e:
 print(f"Error: {e}")

# Create text box and button for user input
text_box = widgets.Text(
value='',
placeholder='Ask about the Indian Penal Code',
description='You:',
disabled=False
)
button = widgets.Button(
description='Ask',
disabled=False,
button_style='',
tooltip='Click to ask a question about IPC',
icon='legal'
)

button.on_click(on_button_click)

# Display the chatbot interface
display(text_box, button)



Error: name 'user_question' is not defined


Text(value='', description='You:', placeholder='Ask about the Indian Penal Code')

Button(description='Ask', icon='legal', style=ButtonStyle(), tooltip='Click to ask a question about IPC')

In [ ]:
!pip install langchain cohere langchain-community google-colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
